# Download Dataset
Databricks job for downloading Petrobras 3W dataset files. This job reads configuration from config.yaml and downloads the 3W dataset according to the specified parameters. Designed to run as a Databricks job.

In [0]:
%pip install uv

In [0]:
%sh uv pip install .

In [0]:
# Get config path from job parameters
dbutils.widgets.text("config_path", "config.yaml")
config_path = dbutils.widgets.get("config_path")

In [0]:
from src.utils import DotConfig
from src.download import DatasetDownloader
config = DotConfig(config_path)

In [0]:
downloader = DatasetDownloader(config)
downloader.download_dataset()